# Carregar e inspecionar dados

O arquivo seguinte foi implementado em um notebook dentro do Google Cloud Plataform, para ser executado dentro do VSCode é necessário fazer autenticação.

In [1]:
# Importanto bibliotecas.
from google.cloud import bigquery
import pandas as pd

In [2]:
# Definindo ID's.
project_id = 'pipeline-etl-ecommerce'
dataset_id = 'etl_ecommerce'
table_id = 'tb_compras'

client = bigquery.Client(project=project_id)

# Selecionando tabela.
query = f"""
    SELECT * FROM `{project_id}.{dataset_id}.{table_id}`
"""

In [3]:
# Transformando tabela em DataFrame.
df = client.query(query).to_dataframe()

# Limpeza e Manipulação dos Dados

In [4]:
# Excluindo colunas que não serão utilizadas para o caso.
df = df.drop(columns=['InvoiceNo', 'StockCode', 'CustomerID'])

In [5]:
# Substituindo valores ausentes da coluna 'Description'.
df['Description'] = df['Description'].replace('?', 'Not informed').fillna('Not informed')

In [6]:
# Transformando data em formato datetime.
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%m/%d/%Y %H:%M')

# Criando colunas de mês e ano.
df['Month'] = df['InvoiceDate'].dt.strftime('%B')
df['Year'] = df['InvoiceDate'].dt.year

In [7]:
# Dropando duplicatas
df = df.drop_duplicates()

In [8]:
# Substituindo valores negativos por 0 na coluna Quantity.
df['Quantity'] = df['Quantity'].apply(lambda x: 0 if x < 0 else x)

# Substituindo valores negativos por 0 na coluna UnitPrice.
df['UnitPrice'] = df['UnitPrice'].apply(lambda x: 0 if x < 0 else x)

In [9]:
# Criando ID.
df = df.reset_index()

In [10]:
clean_table_id = 'tb_compras_clean'
caminho_destino = f'{project_id}.{dataset_id}.{clean_table_id}'

In [11]:
# Exportando tabela.
df.to_gbq(destination_table=caminho_destino, project_id=project_id, if_exists='replace')

<ipython-input-11-52677eeb262d>:2: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=caminho_destino, project_id=project_id, if_exists='replace')
100%|██████████| 1/1 [00:00<00:00, 9554.22it/s]


# Calculando métricas

In [12]:
# Nova tabela
table_id = 'tb_compras_clean'

# Selecionando tabela.
query = f"""
    SELECT * FROM `{project_id}.{dataset_id}.{table_id}`
"""

In [13]:
# Transformando tabela em DataFrame.
df = client.query(query).to_dataframe()

### 1. Receita Total

In [14]:
# Criando coluna com Total de Vendas.
df['TotalSales'] = (df['Quantity'] * df['UnitPrice']).round(2)

total_vendas = round(df['TotalSales'].sum(), 2)
df_total_vendas = pd.DataFrame({'TotalSales': [total_vendas]})
table_id = 'tb_receita'
caminho_destino = f'{project_id}.{dataset_id}.{table_id}'
df_total_vendas.to_gbq(destination_table=caminho_destino, project_id=project_id, if_exists='replace')

<ipython-input-14-e6cb83e547f7>:8: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_total_vendas.to_gbq(destination_table=caminho_destino, project_id=project_id, if_exists='replace')
100%|██████████| 1/1 [00:00<00:00, 3132.42it/s]


### 2. Número de vendas por região

In [15]:
# Calculando número de vendas por país.
vendas_pais = df.groupby('Country')['TotalSales'].sum().reset_index()
table_id = 'tb_vendas_regiao'
caminho_destino = f'{project_id}.{dataset_id}.{table_id}'
vendas_pais.to_gbq(destination_table=caminho_destino, project_id=project_id, if_exists='replace')

<ipython-input-15-5ca279d300df>:5: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  vendas_pais.to_gbq(destination_table=caminho_destino, project_id=project_id, if_exists='replace')
100%|██████████| 1/1 [00:00<00:00, 7206.71it/s]


### 3. Vendas por mês

In [16]:
# Transformando coluna de mês em tipo categórico para ordenação.
ordem_meses = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
df['Month'] = pd.Categorical(df['Month'], categories=ordem_meses, ordered=True)

In [17]:
# Calculando número de vendas por mês/ano.
vendas_mes_ano = df.groupby(['Year', 'Month'])['TotalSales'].sum().reset_index()
table_id = 'tb_vendas_mes_ano'
caminho_destino = f'{project_id}.{dataset_id}.{table_id}'
vendas_mes_ano.to_gbq(destination_table=caminho_destino, project_id=project_id, if_exists='replace')

<ipython-input-17-9c04c684363a>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  vendas_mes_ano = df.groupby(['Year', 'Month'])['TotalSales'].sum().reset_index()
<ipython-input-17-9c04c684363a>:5: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  vendas_mes_ano.to_gbq(destination_table=caminho_destino, project_id=project_id, if_exists='replace')
100%|██████████| 1/1 [00:00<00:00, 9510.89it/s]


### 4. Vendas por categoria de produto

In [18]:
# Calcular a contagem de cada valor na coluna 'Description'.
contagem_description = df['Description'].value_counts().head(10).reset_index()
contagem_description.columns = ['Description', 'Quantity']
table_id = 'tb_vendas_categoria'
caminho_destino = f'{project_id}.{dataset_id}.{table_id}'
contagem_description.to_gbq(destination_table=caminho_destino, project_id=project_id, if_exists='replace')

<ipython-input-18-e08c4e088686>:6: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  contagem_description.to_gbq(destination_table=caminho_destino, project_id=project_id, if_exists='replace')
100%|██████████| 1/1 [00:00<00:00, 6594.82it/s]
